In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from basefactor import fetcher
from basicfunc import *
from localsimulator import *
from dataclean import *

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


# 检查一个完整的模拟流程

## 1. 对比一下数据准确性

In [2]:
gener = Generator('20180101', '20221231')
gener.execute_save()

Folder 2018 created.
Year 2018 items ASHARE saved. shape is (243, 3572)
Year 2018 items CAP saved. shape is (243, 3572)
Year 2018 items OPEN saved. shape is (243, 3572)
Year 2018 items HIGH saved. shape is (243, 3572)
Year 2018 items LOW saved. shape is (243, 3572)
Year 2018 items CLOSE saved. shape is (243, 3572)
Year 2018 items VWAP saved. shape is (0, 0)
Year 2018 items VWAPRET saved. shape is (243, 3572)
Year 2018 items VOL saved. shape is (243, 3572)
Year 2018 items ISZT saved. shape is (243, 3572)
Year 2018 items ISTP saved. shape is (243, 3572)
Year 2018 items WIND01 saved. shape is (243, 3569)
Year 2018 items DAYS saved. shape is (243,)
Year 2018 items STOCKS saved. shape is (3572,)
Year 2018 saved.
Folder 2019 created.
Year 2019 items ASHARE saved. shape is (244, 3770)
Year 2019 items CAP saved. shape is (244, 3769)
Year 2019 items OPEN saved. shape is (244, 3770)
Year 2019 items HIGH saved. shape is (244, 3770)
Year 2019 items LOW saved. shape is (244, 3770)
Year 2019 items C